<a href="https://colab.research.google.com/github/phurisk/Election_Python/blob/main/Election.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#เชื่อม google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#เข้าไป path ใน google drive
import os
os.chdir('/content/drive/Shareddrives')

!pwd #เช็ค path ที่กำลังทำงานอยู่

/content/drive/Shareddrives


In [3]:
import pandas as pd
import os
os.chdir('/content/drive/Shareddrives/Election_Project')

In [4]:

file_name = '/content/drive/Shareddrives/Project_DataViz23data/ectreport66.xlsx' # ชื่อไฟล์ xlsx
dfs = pd.ExcelFile(file_name) # คำสั่งอ่านไฟล์ xlsx

dfs.sheet_names # คำสั่งเช็คว่าในไฟล์ xlsx มี seet ย่อย ในไฟล์ชื่อว่าอะไรบ้าง

['Schema',
 'result_constituencies_PartyList',
 'result_constituencies_Candidate',
 'result_constituencies_status',
 'info_province',
 'info_party_overview',
 'info_constituency',
 'Candidate_Constituency',
 'Candidate_PartyList',
 'Candidate_PM']

In [5]:
# select sheet page by sheet name
Schema = dfs.parse('Schema')
result_constituencies_PartyList = dfs.parse('result_constituencies_PartyList')
result_constituencies_Candidate = dfs.parse('result_constituencies_Candidate')
result_constituencies_status = dfs.parse('result_constituencies_status')
info_province = dfs.parse('info_province')
info_party_overview = dfs.parse('info_party_overview')
info_constituency = dfs.parse('info_constituency')
Candidate_constituency = dfs.parse('Candidate_Constituency')
Candidate_partyList = dfs.parse('Candidate_PartyList')
Candidate_pm = dfs.parse('Candidate_PM')

#Check NaN & Remove NaN (Null)
- result_contituencies_status

In [ ]:
print(result_constituencies_status.isnull().any()) # เช็คค่าว่าง Null
result_constituencies_status.head()

- Fix: result_contituencies_status

In [ ]:
result_constituencies_status = result_constituencies_status.drop(result_constituencies_status.columns[7:18], axis=1)

print(result_constituencies_status.isnull().any())


- info_party_overview

In [26]:
print(info_party_overview.isnull().any()) # เช็คค่าว่าง Null
info_party_overview.head()

id          False
party_no    False
name        False
abbr        False
color       False
logo_url    False
dtype: bool


,id,party_no,name,abbr,color,logo_url
0,701,26.0,ประชาธิปัตย์,ปชป.,#06aff3,https://cdn.ectreport.com/images/logos/ประชาธิ...
1,702,63.0,ประชากรไทย,ปชท.,#0f41ce,https://cdn.ectreport.com/images/logos/ประชากร...
2,703,60.0,ความหวังใหม่,ควม.,#e6b750,https://cdn.ectreport.com/images/logos/ความหวั...
3,705,29.0,เพื่อไทย,พท.,#da3731,https://cdn.ectreport.com/images/logos/เพื่อไท...
4,706,14.0,ชาติพัฒนากล้า,ชพก.,#233e8a,https://cdn.ectreport.com/images/logos/ชาติพัฒ...


In [ ]:
info_party_overview[info_party_overview['party_no'].isnull()] #แสดง row ที่มี party_no ว่าง
# party_no คือ เลขพรรคการเมืองในบัตรเลือกตั้ง ส.ส. บัญชีรายชื่อ
# เติมเลข 99 เพราะไม่มีในรายชื่อสมาชิกสภาผู้แทนราษฎรแบบบัญชีรายชื่อ
# ประชาสามัคคี	- 99
# พลังบูรพา - 99
# รักษ์ธรรม - 99

- Fix:

In [ ]:
# แทนค่าว่างด้วยหมายเลข 99
info_party_overview['party_no'] = info_party_overview['party_no'].fillna(99)

#check
print(info_party_overview.isnull().any()) # เช็คค่าว่าง Null อีกรอบ
info_party_overview.head()

info_constituency

In [ ]:
print(info_constituency.isnull().any()) # เช็คค่าว่าง Null
info_constituency.head()

- Fix: info_constituency

In [27]:
import numpy as np
# แทนค่าว่าง
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'RNG', 'อำเภอเมืองระนอง', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'SBR', 'อำเภอเมืองสิงห์บุรี', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'SKM', 'อำเภอเมืองสมุทรสงคราม', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'TRT', 'อำเภอเมืองตราด', info_constituency['zone'])


print(info_constituency.isnull().any()) # เช็คค่าว่าง Null อีกรอบ
info_constituency.head()

cons_id                False
cons_no                False
prov_id                False
registered_vote        False
total_vote_stations    False
zone                   False
dtype: bool


,cons_id,cons_no,prov_id,registered_vote,total_vote_stations,zone
0,ACR_1,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ
1,ACR_1,1,ACR,145416,310,อำเภอหัวตะพาน
2,ACR_2,2,ACR,155938,331,อำเภอเสนางคนิคม
3,ACR_2,2,ACR,155938,331,อำเภอชานุมาน
4,ACR_2,2,ACR,155938,331,อำเภอปทุมราชวงศา


# Merge Table
- ทำการ Merge table ที่จะใช้

## Table 1

In [13]:
table_merge1 =result_constituencies_Candidate.merge(info_constituency, how ='left', left_on='cons_id',right_on='cons_id')
print(table_merge1.shape)
table_merge1.head()

(13508, 11)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,cons_no,prov_id,registered_vote,total_vote_stations,zone
0,ACR_1,ACR_1_1,9,459,0.43477,706,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ
1,ACR_1,ACR_1_1,9,459,0.43477,706,1,ACR,145416,310,อำเภอหัวตะพาน
2,ACR_1,ACR_1_10,1,38275,36.25419,709,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ
3,ACR_1,ACR_1_10,1,38275,36.25419,709,1,ACR,145416,310,อำเภอหัวตะพาน
4,ACR_1,ACR_1_11,13,126,0.11935,712,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ


In [24]:
result_constituencies_Candidate

,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id
0,ACR_1,ACR_1_1,9,459,0.43477,706
1,ACR_1,ACR_1_10,1,38275,36.25419,709
2,ACR_1,ACR_1_11,13,126,0.11935,712
3,ACR_1,ACR_1_12,12,127,0.12029,742
4,ACR_1,ACR_1_13,11,212,0.20081,761
...,...,...,...,...,...,...
4776,YST_3,YST_3_5,1,37946,36.30085,709
4777,YST_3,YST_3_6,7,821,0.78541,763
4778,YST_3,YST_3_7,9,531,0.50798,712
4779,YST_3,YST_3_8,11,296,0.28317,769


In [25]:
info_constituency

,cons_id,cons_no,prov_id,registered_vote,total_vote_stations,zone
0,ACR_1,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ
1,ACR_1,1,ACR,145416,310,อำเภอหัวตะพาน
2,ACR_2,2,ACR,155938,331,อำเภอเสนางคนิคม
3,ACR_2,2,ACR,155938,331,อำเภอชานุมาน
4,ACR_2,2,ACR,155938,331,อำเภอปทุมราชวงศา
...,...,...,...,...,...,...
1131,YST_2,2,YST,141903,302,"อำเภอป่าติ้ว (ยกเว้นตำบลศรีฐาน, ตำบลกระจาย)"
1132,YST_2,2,YST,141903,302,อำเภอมหาชนะชัย
1133,YST_3,3,YST,146060,303,อำเภอเลิงนกทา
1134,YST_3,3,YST,146060,303,"อำเภอไทยเจริญ (ยกเว้นตำบลน้ำคำ, ตำบลคำไผ่)"


In [14]:
table_merge2 = table_merge1.merge(info_province, how ='left', left_on='prov_id',right_on='prov_id')
print(table_merge2.shape)
table_merge2.head()

(13508, 17)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,cons_no,prov_id,registered_vote,total_vote_stations_x,zone,province_id,province,eng,abbre_thai,total_registered_vote,total_vote_stations_y
0,ACR_1,ACR_1_1,9,459,0.43477,706,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0
1,ACR_1,ACR_1_1,9,459,0.43477,706,1,ACR,145416,310,อำเภอหัวตะพาน,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0
2,ACR_1,ACR_1_10,1,38275,36.25419,709,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0
3,ACR_1,ACR_1_10,1,38275,36.25419,709,1,ACR,145416,310,อำเภอหัวตะพาน,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0
4,ACR_1,ACR_1_11,13,126,0.11935,712,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0


In [15]:

#สร้างตารางตัวแปร data_city โดยเขียนข้อมูลในตารางทีล่ะบรรทัด
data_city = [  {'province':'กาฬสินธุ์',' region':'ตะวันออกเฉียงเหนือ'},
                    {'province':'ขอนแก่น',' region':'ตะวันออกเฉียงเหนือ'},
                    {'province':'ชัยภูมิ',' region':'ตะวันออกเฉียงเหนือ'},
                    {'province':'นครพนม',' region':'ตะวันออกเฉียงเหนือ'},
                    {'province':'นครราชสีมา',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'บึงกาฬ',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'บุรีรัมย์',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'มหาสารคาม',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'มุกดาหาร',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'ยโสธร',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'ร้อยเอ็ด',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'เลย',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'ศรีสะเกษ',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'สกลนคร',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'สุรินทร์',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'หนองคาย',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'หนองบัวลำภู',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'อุบลราชธานี',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'อุดรธานี',' region':'ตะวันออกเฉียงเหนือ'},
                  {'province':'อำนาจเจริญ',' region':'ตะวันออกเฉียงเหนือ'},

    ###แถว 2
                    {'province':'เชียงใหม่',' region':'เหนือ'},
                  {'province':'เชียงราย',' region':'เหนือ'},
                  {'province':'น่าน',' region':'เหนือ'},
                  {'province':'พะเยา',' region':'เหนือ'},
                  {'province':'แพร่',' region':'เหนือ'},
                  {'province':'แม่ฮ่องสอน',' region':'เหนือ'},
                  {'province':'ลำปาง',' region':'เหนือ'},
                  {'province':'ลำพูน',' region':'เหนือ'},
                  {'province':'อุตรดิตถ์',' region':'เหนือ'},


    ###แถว 3
                    {'province':'อุทัยธานี',' region':'กลาง'},
                  {'province':'ชัยนาท',' region':'กลาง'},
                  {'province':'สิงห์บุรี',' region':'กลาง'},
                  {'province':'อ่างทอง',' region':'กลาง'},
                  {'province':'พระนครศรีอยุธยา',' region':'กลาง'},
                  {'province':'ลพบุรี',' region':'กลาง'},
                  {'province':'สระบุรี',' region':'กลาง'},
                  {'province':'นครนายก',' region':'กลาง'},
                 {'province':'สุโขทัย',' region':'กลาง'},
                  {'province':'กรุงเทพมหานคร',' region':'กลาง'},
                  {'province':'นนทบุรี',' region':'กลาง'},
                  {'province':'ปทุมธานี',' region':'กลาง'},
                  {'province':'สมุทรปราการ',' region':'กลาง'},
                  {'province':'สมุทรสาคร',' region':'กลาง'},
                  {'province':'สมุทรสงคราม',' region':'กลาง'},
                  {'province':'นครปฐม',' region':'กลาง'},
                  {'province':'นครสวรรค์',' region':'กลาง'},
                  {'province':'สุพรรณบุรี',' region':'กลาง'},
                  {'province':'เพชรบูรณ์',' region':'กลาง'},
                    {'province':'พิจิตร',' region':'กลาง'},
                  {'province':'พิษณุโลก',' region':'กลาง'},
                  {'province':'กำแพงเพชร',' region':'กลาง'},

    ###แถว
                    {'province':'ชุมพร',' region':'ใต้'},
                  {'province':'นครศรีธรรมราช',' region':'ใต้'},
                  {'province':'นราธิวาส',' region':'ใต้'},
                  {'province':'ปัตตานี',' region':'ใต้'},
                  {'province':'พัทลุง',' region':'ใต้'},
                  {'province':'สงขลา',' region':'ใต้'},
                  {'province':'สุราษฎร์ธานี',' region':'ใต้'},
                  {'province':'ยะลา',' region':'ใต้'},
                  {'province':'กระบี่',' region':'ใต้'},
                  {'province':'พังงา',' region':'ใต้'},
                  {'province':'ภูเก็ต',' region':'ใต้'},
                  {'province':'ระนอง',' region':'ใต้'},
                  {'province':'สตูล',' region':'ใต้'},
                  {'province':'ตรัง',' region':'ใต้'},

     ### แถว5
                    {'province':'จันทบุรี',' region':'ตะวันออก'},
                    {'province':'ฉะเชิงเทรา',' region':'ตะวันออก'},
                    {'province':'ชลบุรี',' region':'ตะวันออก'},
                    {'province':'ตราด',' region':'ตะวันออก'},
                    {'province':'ปราจีนบุรี',' region':'ตะวันออก'},
                    {'province':'ระยอง',' region':'ตะวันออก'},
                    {'province':'สระแก้ว',' region':'ตะวันออก'},

   ### แถว6
                   {'province':'กาญจนบุรี',' region':'ตะวันตก'},
                   {'province':'ตาก',' region':'ตะวันตก'},
                    {'province':'ประจวบคีรีขันธ์',' region':'ตะวันตก'},
                   {'province':'เพชรบุรี',' region':'ตะวันตก'},
                   {'province':'ราชบุรี',' region':'ตะวันตก'}]




In [16]:
Region = pd.DataFrame(data_city)

In [17]:
F1 = table_merge2.merge(Region, how ='left', left_on='province',right_on='province')
print(F1.shape)
F1.head()

(13508, 18)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,cons_no,prov_id,registered_vote,total_vote_stations_x,zone,province_id,province,eng,abbre_thai,total_registered_vote,total_vote_stations_y,region
0,ACR_1,ACR_1_1,9,459,0.43477,706,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0,ตะวันออกเฉียงเหนือ
1,ACR_1,ACR_1_1,9,459,0.43477,706,1,ACR,145416,310,อำเภอหัวตะพาน,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0,ตะวันออกเฉียงเหนือ
2,ACR_1,ACR_1_10,1,38275,36.25419,709,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0,ตะวันออกเฉียงเหนือ
3,ACR_1,ACR_1_10,1,38275,36.25419,709,1,ACR,145416,310,อำเภอหัวตะพาน,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0,ตะวันออกเฉียงเหนือ
4,ACR_1,ACR_1_11,13,126,0.11935,712,1,ACR,145416,310,อำเภอเมืองอำนาจเจริญ,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0,ตะวันออกเฉียงเหนือ


## Table 2

In [18]:
F2 = result_constituencies_status.merge(info_province, how ='left', left_on='prov_id',right_on='prov_id')
print(F2.shape)
F2.head()

(400, 13)


,prov_id,cons_id,turn_out,percent_turn_out,counted_vote_stations,percent_count,pause_report,province_id,province,eng,abbre_thai,total_registered_vote,total_vote_stations
0,ACR,ACR_1,105574,72.60136,310,100.0,False,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0
1,ACR,ACR_2,111813,71.70350,331,100.0,False,37.0,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641.0
2,ATG,ATG_1,89756,80.19872,239,100.0,False,15.0,อ่างทอง,ANG THONG,อท,224656,492.0
3,ATG,ATG_2,91466,81.13075,253,100.0,False,15.0,อ่างทอง,ANG THONG,อท,224656,492.0
4,AYA,AYA_1,110317,79.55591,226,100.0,False,14.0,พระนครศรีอยุธยา,PHRA NAKHON SI AYUTTHAYA,อย,664431,1060.0


## Table 3

In [30]:
tableM3 = pd.merge(result_constituencies_Candidate, Candidate_constituency, on='mp_app_id', how='left')

In [31]:
F3 = pd.merge(tableM3, info_party_overview, left_on='mp_app_party_id',right_on='id', how='left')

In [ ]:
F3

#Check Null
- เช็ค datafram ที่ merge แล้วว่ามี null หรือไม่ (ถ้า true จะเป็นค่า null)

In [19]:
print(F1.isnull().any())

cons_id                  False
mp_app_id                False
mp_app_rank              False
mp_app_vote              False
mp_app_vote_percent      False
party_id                 False
cons_no                  False
prov_id                  False
registered_vote          False
total_vote_stations_x    False
zone                     False
province_id              False
province                 False
eng                      False
abbre_thai               False
total_registered_vote    False
total_vote_stations_y    False
 region                  False
dtype: bool


In [20]:
print(F2.isnull().any())

prov_id                  False
cons_id                  False
turn_out                 False
percent_turn_out         False
counted_vote_stations    False
percent_count            False
pause_report             False
province_id              False
province                 False
eng                      False
abbre_thai               False
total_registered_vote    False
total_vote_stations      False
dtype: bool


#นำออกเป็น CSV File
- โดยจะไปที่ drive ที่ตั้งไว้ตั้งแต่ตอนแรก

In [21]:
# F1.to_csv('F1.csv', index=False)

In [22]:
# F2.to_csv('F2.csv', index=False)